Verifying the list of workstation(ip, t) made in splunk is correct.

dhcp_sample.csv ... input from a Splunk search to find all the trooper workstation leases
```
`renew` troopers | reverse| rename _time as lease_time, dest_nt_host as workstation 
| table lease_time workstation dest_ip | sort workstation lease_time
```
ws_leases.csv ..... output of the workstation leases found using this list. 

This work verified that the splunk results matched these results for 

In [66]:
import pandas as pd
from operator import itemgetter
import pdb

def tell(s):
    print(s+': \n', eval(s))

def prt(series):
    for item in list(series):
        print(item)
        
# Sample data: all the dhcp leases for the period
# sample_file = 'dhcp_sample_09212020.csv'
sample_file = 'dhcp_sample_09172020.csv'
# Splunk results for the same period
# splunk_file = 'ws_splunk_0921202.csv'
splunk_file = 'ws_splunk_09172020.csv'


# Download all the dhcp renews
dhcp = pd.read_csv(sample_file)

# Make new column with the current index
dhcp['seq'] = dhcp.index


In [67]:
# Reorder the columns
dhcp.reindex(columns=['workstation', 'dest_ip', 'seq', 'lease_time'])

# Assemble the separate columns into a list in it's own column
dhcp['events'] = dhcp[['dest_ip','seq','lease_time']].values.tolist()

# Make a new df of the interesting column
df = dhcp[['workstation', 'events']]


In [68]:
def remove_redundant_leases(events):
    # Remove the redundant dhcp leases for this workstation, 
    # the ones that do not repeat the current ip assignement.
    
    # events is a Series of lists: dest_ip, seq, lease_time. 
    # result will be a list of lists
    
    dest_ip, seq, lease_time = 0, 1, 2
    seq = 2
    lease_time =2
    
    # sort the events for the workstation by sequence, seq
    s = sorted(list(events), key = itemgetter(seq))

    # Accumulate the meaningful leases. Start with the first
    lease =  [[s[0][dest_ip],s[0][lease_time]]]
    
    # If there is only one lease then we are done
    
    if len(s)>1:
    
        # Do the rest
        for i in range(1, len(s)):

            # Add lease only if it is not the same as the last lease
            # Note: A lease is not redundant if the workstation 
            # received an intermediate ip address.

            if s[i][dest_ip] != s[i-1][dest_ip]:

                lease.append([s[i][dest_ip], s[i][lease_time]])
            
    return lease 
        
    

In [69]:

def xremove_redundant_leases(events):
    # Remove the redundant dhcp leases for this workstation, 
    # the ones that do not repeat the current ip assignement.
    
    # events is a Series of lists: dest_ip, seq, lease_time. 
    # result will be a list of lists
    
    idest_ip, iseq, ilease_time = 0, 1, 2
    
    # sort the events for the workstation by sequence, seq
    s = sorted(list(events), key = itemgetter(iseq))

    dest_ip = lambda i: s[i][idest_ip]
    seq = lambda i: s[i][iseq]
    lease_time = lambda i: s[i][ilease_time]

    # Accumulate the meaningful leases. Start with the first
    lease =  [dest_ip(0), lease_time(0)]
    # If there is only one lease then we are done
    
    if len(s)>1:
    
        # Do the rest
        # Add lease only if it is not the same as the last lease
        # Note: A lease is not redundant if the workstation 
        # received an intermediate ip address.

        for i in range(1, len(s)):

            if dest_ip(i) != dest_ip(i-1):
                lease.append([dest_ip(i), lease_time(i)])
                
        return lease 

In [70]:

def yyremove_redundant_leases(events):
    # Remove the redundant dhcp leases for this workstation, 
    # the ones that do not repeat the current ip assignement.
    
    # events is a Series of lists: dest_ip, seq, lease_time. 
    # result will be a list of lists
    idest_ip, iseq, ilease_time = 0, 1, 2
    
    # sort the events for the workstation by sequence, seq
    s = sorted(list(events), key = itemgetter(iseq))
    print('s:')
    prt(s)

    dest_ip    = lambda i: s[i][idest_ip]
    seq        = lambda i: s[i][iseq]
    lease_time = lambda i: s[i][ilease_time]
#     lease      = lambda i: [ lease_time(i), dest_ip(i) ]
    lease      = lambda i: [ lease_time(i), dest_ip(i) ]
    
    stack      = [] ; result = []
    push       = lambda x: stack.append(x)
    jump       = lambda x: stack.insert(0, x)
    pop        = lambda  : stack.pop()
    top        = lambda  : stack[-1]
    bottom     = lambda  : stack.pop(0)
    size       = lambda  : len(stack)
    empty      = lambda  : size() == 0
#     pdb.set_trace()
    # Hold the first in the stack to eliminate redundant leases
    push(lease(0))

    # Do the rest, if any.
    for i in range(1, len(s)):

        # If time is the same as on top of stack, replace it with new lease
        if lease_time(i) == top()[0]:
            pop()
            push(lease(i))

        # When the dest_ip changes save stacked lease. 
        # Hold current one to eliminate redundant leases.
        
        elif dest_ip(i) != top()[1]:
            result.append(pop())
            push(lease(i))
                
    # Empty the stack into the output orderd by increasing lease_time
    while not empty():
        result.append(bottom())
    
    return result

In [80]:

def yremove_redundant_leases(events):
    # Remove the redundant dhcp leases for this workstation, 
    # the ones that do not repeat the current ip assignement.
    
    # events is a Series of lists: dest_ip, seq, lease_time. 
    # result will be a list of lists
    idest_ip, iseq, ilease_time = 0, 1, 2
    
    # sort the events for the workstation by sequence, seq
    s = sorted(list(events), key = itemgetter(iseq))
    print('s:')
    prt(s)

    dest_ip    = lambda i: s[i][idest_ip]
    seq        = lambda i: s[i][iseq]
    lease_time = lambda i: s[i][ilease_time]
#     lease      = lambda i: [ lease_time(i), dest_ip(i) ]
    lease      = lambda i: [ lease_time(i), dest_ip(i) ]
    
    stack      = [] ; result = []
    push       = lambda x: stack.append(x)
    jump       = lambda x: stack.insert(0, x)
    pop        = lambda  : stack.pop()
    top        = lambda  : stack[-1]
    bottom     = lambda  : stack.pop(0)
    size       = lambda  : len(stack)
    empty      = lambda  : size() == 0
#     pdb.set_trace()
    # Hold the first in the stack to eliminate redundant leases
    push(lease(0))

    # Do the rest, if any.
    for i in range(1, len(s)):

        # If time is the same as on top of stack, replace it with new lease
        if lease_time(i) == top()[0]:
            pop()
            push(lease(i))

        # When the dest_ip changes save stacked lease. 
        # Hold current one to eliminate redundant leases.
        
        elif dest_ip(i) != top()[1]:
#             result.append(pop())
            push(lease(i))
                
    # Return the stack
    
    return stack

In [81]:
# Make a dataframe and output to csv


new = []
for i, ws in enumerate(df.workstation.unique()):
    print('{} : {}'.format(i, ws))
    leases = yremove_redundant_leases(df[df.workstation==ws].events)
    print('leases:')
    prt(leases)
    for lease in leases:

        lease.append(ws)
        new.append(lease)
#         prt(lease)

0 : D-MXL8401ZTR.troopers.state.nyenet
s:
['10.81.136.81', 0, 1600315211]
['10.81.136.81', 5963, 1600358411]
leases:
[1600315211, '10.81.136.81']
1 : D-2UA4130KFH.troopers.state.nyenet
s:
['10.81.136.157', 1, 1600315222]
['10.81.136.157', 5964, 1600358422]
leases:
[1600315222, '10.81.136.157']
2 : L-CNU312C4JM.troopers.state.nyenet
s:
['10.81.6.58', 2, 1600315227]
['10.81.6.58', 349, 1600317027]
['10.81.6.58', 716, 1600318828]
['10.81.6.58', 1005, 1600320628]
['10.81.6.58', 1207, 1600322428]
['10.81.6.58', 1518, 1600324229]
['10.81.6.58', 1651, 1600326029]
['10.81.6.58', 1810, 1600327829]
['10.81.6.58', 2024, 1600329630]
['10.81.6.58', 2188, 1600331430]
['10.81.6.58', 2296, 1600333230]
['10.81.6.58', 2421, 1600335031]
['10.81.6.58', 2533, 1600336831]
['10.81.6.58', 2666, 1600338631]
['10.81.6.58', 2828, 1600340431]
['10.81.6.58', 3049, 1600342232]
['10.81.6.58', 3296, 1600344032]
['10.81.6.58', 3550, 1600345832]
['10.81.6.58', 3838, 1600347633]
['10.81.6.58', 4097, 1600349433]
['10.81.

['10.81.6.54', 3703, 1600346818]
['10.81.6.54', 3797, 1600347457]
['10.81.6.54', 4066, 1600349258]
['10.81.6.54', 4335, 1600351058]
['10.81.6.54', 4945, 1600352858]
['10.81.6.54', 5161, 1600354659]
['10.81.6.54', 5447, 1600356459]
['10.81.6.54', 5494, 1600356667]
['10.81.6.54', 5973, 1600358470]
['10.81.6.54', 6308, 1600360274]
['10.81.6.54', 6706, 1600362079]
['10.81.6.54', 7005, 1600363875]
['10.81.6.54', 7296, 1600365676]
['10.81.6.54', 7640, 1600367476]
['10.81.6.54', 7820, 1600369277]
['10.81.6.54', 8008, 1600371077]
['10.81.6.54', 8206, 1600372877]
['10.81.6.54', 8360, 1600374678]
['10.81.6.54', 8457, 1600376478]
['10.81.6.54', 8562, 1600378279]
['10.81.6.54', 8642, 1600380079]
['10.81.6.54', 8746, 1600381879]
['10.81.222.70', 8839, 1600383539]
['10.81.6.54', 8849, 1600383680]
['10.81.6.54', 8976, 1600385480]
['10.81.6.54', 9138, 1600387280]
['10.81.6.54', 9326, 1600389081]
['10.81.6.54', 9536, 1600390881]
['10.81.6.54', 9740, 1600392681]
['10.81.6.54', 10026, 1600394482]
['10.81

s:
['10.81.170.154', 239, 1600316439]
['10.81.170.154', 6212, 1600359640]
leases:
[1600316439, '10.81.170.154']
222 : W2UA8520SK3.troopers.state.nyenet
s:
['10.81.8.143', 240, 1600316450]
leases:
[1600316450, '10.81.8.143']
223 : D-MXL837240Z.troopers.state.nyenet
s:
['10.81.242.92', 241, 1600316452]
['10.81.242.92', 6213, 1600359651]
leases:
[1600316452, '10.81.242.92']
224 : D-2UA2260L5N.troopers.state.nyenet
s:
['10.81.217.6', 242, 1600316457]
['10.81.217.6', 6215, 1600359658]
leases:
[1600316457, '10.81.217.6']
225 : D-2UA4130L10.troopers.state.nyenet
s:
['10.81.34.51', 243, 1600316457]
leases:
[1600316457, '10.81.34.51']
226 : D-2UA4130L3K.troopers.state.nyenet
s:
['10.81.181.98', 244, 1600316468]
['10.81.181.98', 2510, 1600336398]
['10.81.181.98', 2538, 1600336919]
['10.81.181.98', 2692, 1600338900]
['10.81.181.98', 2693, 1600338919]
['10.81.181.98', 2703, 1600339036]
['10.81.181.98', 2708, 1600339105]
['10.81.181.98', 2712, 1600339153]
['10.81.181.98', 2715, 1600339175]
['10.81.

leases:
[1600317141, '10.81.217.14']
331 : D-2UA41225KY.troopers.state.nyenet
s:
['10.81.74.42', 370, 1600317147]
leases:
[1600317147, '10.81.74.42']
332 : D-MXL81523FC.troopers.state.nyenet
s:
['10.81.97.159', 371, 1600317160]
['10.81.97.159', 6323, 1600360360]
leases:
[1600317160, '10.81.97.159']
333 : D-2UA41222QP.troopers.state.nyenet
s:
['10.81.97.179', 372, 1600317160]
['10.81.97.179', 6324, 1600360360]
leases:
[1600317160, '10.81.97.179']
334 : D-2UA4121T95.troopers.state.nyenet
s:
['10.81.157.177', 375, 1600317178]
leases:
[1600317178, '10.81.157.177']
335 : D-2UA4121T9V.troopers.state.nyenet
s:
['10.81.157.159', 376, 1600317178]
leases:
[1600317178, '10.81.157.159']
336 : L-CNU312C3JR.troopers.state.nyenet
s:
['10.81.26.30', 378, 1600317195]
['10.81.26.30', 6331, 1600360397]
leases:
[1600317195, '10.81.26.30']
337 : ARAH-0044.troopers.state.nyenet
s:
['10.81.197.30', 379, 1600317203]
['10.81.197.30', 6335, 1600360422]
['10.81.197.30', 6336, 1600360422]
leases:
[1600317203, '10

s:
['10.81.88.76', 523, 1600318053]
['10.81.88.76', 6493, 1600361253]
leases:
[1600318053, '10.81.88.76']
452 : D-2UA4121T21.troopers.state.nyenet
s:
['10.81.36.89', 524, 1600318059]
['10.81.36.89', 6494, 1600361259]
leases:
[1600318059, '10.81.36.89']
453 : D-2UA4121TGJ.troopers.state.nyenet
s:
['10.81.136.206', 525, 1600318061]
['10.81.136.206', 6495, 1600361261]
leases:
[1600318061, '10.81.136.206']
454 : L-CNU4139JTD.troopers.state.nyenet
s:
['10.81.178.60', 526, 1600318062]
['10.81.178.60', 6496, 1600361263]
leases:
[1600318062, '10.81.178.60']
455 : L-3DT8RC2.troopers.state.nyenet
s:
['10.81.116.79', 527, 1600318066]
['10.81.116.79', 6497, 1600361266]
leases:
[1600318066, '10.81.116.79']
456 : L-F2S63G2.troopers.state.nyenet
s:
['10.81.158.178', 528, 1600318090]
['10.81.158.178', 6500, 1600361290]
leases:
[1600318090, '10.81.158.178']
457 : L-BN24NQ2.troopers.state.nyenet
s:
['10.81.136.112', 529, 1600318091]
['10.81.136.112', 6501, 1600361291]
leases:
[1600318091, '10.81.136.112

...

s:
['10.81.27.42', 5420, 1600356326]
['10.81.27.42', 10831, 1600399526]
leases:
[1600356326, '10.81.27.42']
4017 : D-2UA4130L06.troopers.state.nyenet
s:
['10.81.116.34', 5421, 1600356327]
['10.81.116.34', 10832, 1600399527]
leases:
[1600356327, '10.81.116.34']
4018 : D-2UA4130L35.troopers.state.nyenet
s:
['10.81.146.163', 5422, 1600356336]
['10.81.146.163', 5423, 1600356336]
['10.81.146.163', 10833, 1600399536]
leases:
[1600356336, '10.81.146.163']
4019 : D-2UA4130H1N.troopers.state.nyenet
s:
['10.81.65.26', 5424, 1600356344]
['10.81.65.26', 10834, 1600399545]
leases:
[1600356344, '10.81.65.26']
4020 : D-2UA7191YB7.troopers.state.nyenet
s:
['10.81.242.65', 5425, 1600356346]
['10.81.242.65', 10835, 1600399546]
leases:
[1600356346, '10.81.242.65']
4021 : D-2UA4121T41.troopers.state.nyenet
s:
['10.81.19.68', 5426, 1600356351]
['10.81.19.68', 10836, 1600399551]
leases:
[1600356351, '10.81.19.68']
4022 : L-G8C7QN2.troopers.state.nyenet
s:
['10.81.114.36', 5427, 1600356357]
['10.81.114.36', 

[1600356774, '10.81.18.131']
4146 : L-7Q85NQ2.troopers.state.nyenet
s:
['10.81.36.161', 5587, 1600356783]
['10.81.36.161', 10972, 1600399983]
leases:
[1600356783, '10.81.36.161']
4147 : D-2UA312076J.troopers.state.nyenet
s:
['10.81.18.219', 5589, 1600356784]
['10.81.18.219', 10973, 1600399985]
leases:
[1600356784, '10.81.18.219']
4148 : L-3BZXMQ2.troopers.state.nyenet
s:
['10.81.27.106', 5590, 1600356785]
['10.81.27.106', 10974, 1600399985]
leases:
[1600356785, '10.81.27.106']
4149 : L-JXSCPN2.troopers.state.nyenet
s:
['10.81.136.126', 5591, 1600356785]
['10.81.136.126', 10976, 1600399985]
leases:
[1600356785, '10.81.136.126']
4150 : D-MXL8401YGQ.troopers.state.nyenet
s:
['10.81.217.42', 5592, 1600356785]
['10.81.217.42', 10975, 1600399985]
leases:
[1600356785, '10.81.217.42']
4151 : L-8XFTMQ2.troopers.state.nyenet
s:
['10.81.136.40', 5593, 1600356787]
['10.81.136.40', 10977, 1600399987]
leases:
[1600356787, '10.81.136.40']
4152 : D-MXL8401ZY9.troopers.state.nyenet
s:
['10.81.11.131', 

s:
['10.81.178.42', 5743, 1600357161]
['10.81.178.42', 11110, 1600400362]
leases:
[1600357161, '10.81.178.42']
4272 : L-CNU412DDK9.troopers.state.nyenet
s:
['10.81.209.29', 5744, 1600357164]
leases:
[1600357164, '10.81.209.29']
4273 : ARAH-0047.troopers.state.nyenet
s:
['10.81.174.26', 5745, 1600357171]
['10.81.174.26', 11111, 1600400370]
leases:
[1600357171, '10.81.174.26']
4274 : L-CND1153B28.troopers.state.nyenet
s:
['10.81.178.39', 5746, 1600357178]
['10.81.178.39', 11112, 1600400379]
leases:
[1600357178, '10.81.178.39']
4275 : L-CNU312C426.troopers.state.nyenet
s:
['10.81.19.130', 5747, 1600357178]
['10.81.19.130', 11113, 1600400379]
leases:
[1600357178, '10.81.19.130']
4276 : L-CNU312C43J.troopers.state.nyenet
s:
['10.81.19.65', 5748, 1600357179]
['10.81.19.65', 11114, 1600400380]
leases:
[1600357179, '10.81.19.65']
4277 : D-2UA44010KH.troopers.state.nyenet
s:
['10.81.153.156', 5749, 1600357183]
leases:
[1600357183, '10.81.153.156']
4278 : L-JXBQ5H2.troopers.state.nyenet
s:
['10.

['10.81.72.35', 11243, 1600401233]
leases:
[1600358033, '10.81.72.35']
4398 : D-2UA4130KFP.troopers.state.nyenet
s:
['10.81.72.110', 5905, 1600358055]
['10.81.72.110', 11245, 1600401255]
leases:
[1600358055, '10.81.72.110']
4399 : D-2UA1201Y7Q.troopers.state.nyenet
s:
['10.81.41.28', 5906, 1600358065]
['10.81.41.28', 11246, 1600401265]
leases:
[1600358065, '10.81.41.28']
4400 : E65407500c.troopers.state.nyenet
s:
['10.81.19.112', 5907, 1600358081]
['10.81.19.112', 11247, 1600401282]
leases:
[1600358081, '10.81.19.112']
4401 : D-MXL8152461.troopers.state.nyenet
s:
['10.81.163.157', 5908, 1600358082]
leases:
[1600358082, '10.81.163.157']
4402 : D-MXL837240S.troopers.state.nyenet
s:
['10.81.242.111', 5909, 1600358083]
['10.81.242.111', 11248, 1600401282]
leases:
[1600358083, '10.81.242.111']
4403 : D-2UA4130GZ1.troopers.state.nyenet
s:
['10.81.82.154', 5910, 1600358091]
['10.81.82.154', 11250, 1600401291]
leases:
[1600358091, '10.81.82.154']
4404 : D-2UA5192DVK.troopers.state.nyenet
s:
['

s:
['10.81.191.161', 7114, 1600364653]
leases:
[1600364653, '10.81.191.161']
4505 : D-2UA4121THB.troopers.state.nyenet
s:
['10.81.148.131', 7122, 1600364692]
['10.81.148.131', 7753, 1600368640]
['10.81.148.131', 7780, 1600368989]
['10.81.148.131', 7790, 1600369043]
leases:
[1600364692, '10.81.148.131']
4506 : D-2UA23916C5.troopers.state.nyenet
s:
['10.81.168.111', 7125, 1600364710]
['10.81.168.111', 7206, 1600365368]
leases:
[1600364710, '10.81.168.111']
4507 : minint-0bld6n2.troopers.state.nyenet
s:
['10.81.148.131', 7141, 1600364866]
leases:
[1600364866, '10.81.148.131']
4508 : L-61K1NQ2.troopers.state.nyenet
s:
['10.81.33.34', 7161, 1600365053]
leases:
[1600365053, '10.81.33.34']
4509 : minint-v7go4jb.troopers.state.nyenet
s:
['10.81.148.131', 7172, 1600365133]
leases:
[1600365133, '10.81.148.131']
4510 : D-2UA4130K95.troopers.state.nyenet
s:
['10.81.207.34', 7184, 1600365236]
leases:
[1600365236, '10.81.207.34']
4511 : D-2UA4130L1W.troopers.state.nyenet
s:
['10.81.148.77', 7194, 16

In [72]:
cols = ['lease_time', 'dest_ip', 'workstation']
result = pd.DataFrame(new, columns=cols)
result = result.sort_values(by=['workstation', 'lease_time', 'dest_ip'], ascending=[True, True, True], ignore_index=True)
result.to_csv('ws_leases.csv', index=False)

In [82]:
result[result['workstation']=="TROOPKPC.troopers.state.nyenet"]

,lease_time,dest_ip,workstation
4506,1600363894,10.81.208.106,TROOPKPC.troopers.state.nyenet
4507,1600364259,10.81.208.107,TROOPKPC.troopers.state.nyenet
4508,1600367698,10.81.208.106,TROOPKPC.troopers.state.nyenet


In [75]:
# Are the results of this calculation, result, the same as splunk, ws?
# Sort them the same
WS = ws.sort_values(by=['dest_ip', 'lease_time', 'workstation'], ignore_index=True)
RESULT = result.sort_values(by=['dest_ip', 'lease_time', 'workstation'], ignore_index=True)

WS.equals(RESULT)


False

In [74]:
# Read the Splunk results for workstations
ws = pd.read_csv(splunk_file)

In [76]:
print(WS)
print(RESULT)

      lease_time       dest_ip                         workstation
0     1600315525   10.77.16.62   W2UA7061S1S.troopers.state.nyenet
1     1600315743    10.81.1.60  D-2UA4121THZ.troopers.state.nyenet
2     1600348268    10.81.1.69  D-MXL8421ZLM.troopers.state.nyenet
3     1600342457  10.81.10.100  D-2UA3120Y5N.troopers.state.nyenet
4     1600346633  10.81.10.101  D-MXL8431MX3.troopers.state.nyenet
...          ...           ...                                 ...
4630  1600357117  10.81.99.219  D-2UA41222XW.troopers.state.nyenet
4631  1600318450  10.81.99.220  D-2UA4130H07.troopers.state.nyenet
4632  1600348667  10.81.99.221  D-2UA3120Y28.troopers.state.nyenet
4633  1600327397  10.81.99.222   MFP13575575.troopers.state.nyenet
4634  1600340248   10.81.99.34     NPI24FB18.troopers.state.nyenet

[4635 rows x 3 columns]
      lease_time       dest_ip                         workstation
0     1600315525   10.77.16.62   W2UA7061S1S.troopers.state.nyenet
1     1600315743    10.81.1.60  D-2UA

In [77]:
WS == RESULT

ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:
"""
#########################################################################
# Concat _splunk onto the names of the columns

ws.columns = [ 'dest_ip_splunk', 'workstation_splunk', 'lease_time_splunk' ]


# Compare the two df for differences

# First do a left join on workstation

compare = result.merge(ws, how='left', \
                       left_on='workstation', \
                       right_on='workstation_splunk').reset_index()
# right join
# compare = ws.merge(result, how='right', \
#                        left_on='workstation_splunk', \
#                        right_on='workstation').reset_index()
compare[compare('workstation']=='L-CNU312C4JM.troopers.state.nyenet'

# result.equals(ws)
# compare.head(10)

# Look for any values that are not the same. If no rows appear, then they are the same.

compare[ (compare.dest_ip!=compare.dest_ip_splunk) \
        | (compare.workstation!=compare.workstation_splunk) \
        | (compare.lease_time!=compare.lease_time_splunk) ]
compare[compare['workstation']=='L-CNU312C4JM.troopers.state.nyenet']
# compare.head()

df1 = compare[['dest_ip', 'workstation', 'lease_time']]
df2 = compare[['dest_ip_splunk', 'workstation_splunk', 'lease_time_splunk']]
df2.columns=['dest_ip', 'workstation', 'lease_time']
df1.equals(df2)

df1.head()

df2.head()

# T =[[0,'barry',95],[1,'dennis',80],[2,'dennis',96],[3,'cheryl',75]]
# S = sorted(T,key=itemgetter(1), reverse=False)
# print(list(S))
# SS = sorted(S, key=itemgetter(2), reverse=True)
# print(list(SS))
# sorted(T, key=itemgetter(1,2))
"""

In [24]:
from datetime import datetime, date
import time
d=date(2020,10,5)
f'{d}'
x = time.mktime(d.timetuple())
f'{x}'



'1601870400.0'